In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 403.4/488.0 MB 132.9 MB/s eta 0:00:01
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    tf-nightly from https://files.pythonhosted.org/packages/a9/c6/1737040ebeddf62fdf7b8875e7006efbc5e7583b461521b39392dc62ffd3/tf_nightly-2.14.0.dev20230524-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl:
        Expected sha256 8a7b717e241221510af8a9d888879d164845b495e6becbc4442ab0bd669fe263
             Got        ab3aa2c6ecc102741e7c2a70c5e68170c9ed3775a6e695472f3035d37d232e67

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.12.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-05-24 11:52:53--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.006s  

2023-05-24 11:52:53 (56.0 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-05-24 11:52:53--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
train = pd.read_csv(train_file_path, sep="\t", names=["type", "msg"])
test = pd.read_csv(test_file_path, sep="\t", names=["type", "msg"])

In [ ]:
train.dropna()
test.dropna()

,type,msg
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
train["type"] = pd.factorize(train["type"])[0]
test["type"] = pd.factorize(test["type"])[0]

In [ ]:
train_labels =  train["type"].values
test_labels =  test["type"].values

In [ ]:
vocab_size=1024
maxwords=48
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=maxwords)

In [ ]:
vectorizer.adapt(train['msg'])
train_vectorized=vectorizer(train['msg'])
print(train_vectorized)

tf.Tensor(
[[  1   1   1 ...   0   0   0]
 [  4  31 264 ...   0   0   0]
 [ 22   7 785 ...   0   0   0]
 ...
 [ 49 450 313 ...   0   0   0]
 [118  89   1 ...   0   0   0]
 [ 94  12 404 ...   0   0   0]], shape=(4179, 48), dtype=int64)


In [ ]:
model=keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size,128,input_length=maxwords))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)))
model.add(keras.layers.Dropout(0.2))  # Dropout regularization
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128,return_sequences=True)))
model.add(keras.layers.Dropout(0.2))  # Dropout regularization
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(keras.layers.Dropout(0.2))  # Dropout regularization
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 512, input_length=maxwords))
model.add(keras.layers.SimpleRNN(128, return_sequences=True))
model.add(keras.layers.Dropout(0.2))  # Dropout regularization
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128)))
model.add(keras.layers.Dropout(0.2))  # Dropout regularization  # Dropout regularization
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout regularization
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(train_vectorized.numpy(),train_labels, test_size=0.8)

In [ ]:
stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test),callbacks=[stop])

Epoch 1/10
27/27 [==============================] - 14s 511ms/step - loss: 0.0253 - accuracy: 0.9940 - val_loss: 0.1674 - val_accuracy: 0.9644
Epoch 2/10
27/27 [==============================] - 14s 514ms/step - loss: 0.0207 - accuracy: 0.9964 - val_loss: 0.2132 - val_accuracy: 0.9602
Epoch 3/10
27/27 [==============================] - 16s 615ms/step - loss: 0.0226 - accuracy: 0.9976 - val_loss: 0.1922 - val_accuracy: 0.9647
Epoch 3: early stopping


In [ ]:
test_loss, test_acc = model.evaluate(vectorizer(test['msg']),test_labels, verbose=2)
print('Model accuracy on validation data: ',test_acc)

44/44 - 4s - loss: 0.1371 - accuracy: 0.9626 - 4s/epoch - 89ms/step
Model accuracy on validation data:  0.9626436829566956


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text_vectorized = vectorizer([pred_text])
  prediction = model.predict(pred_text_vectorized)[0][0]
  label = "spam" if prediction >= 0.5 else "ham"
  return [prediction, label]
pred_text = "how are you doing today"
prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 30ms/step
[0.0032448377, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 47ms/step
You passed the challenge. Great job!
